In [16]:
import numpy as np
import pandas as pd
from scipy.optimize import Bounds
from copy import copy
from scipy.optimize import OptimizeResult, minimize
from optimparallel import minimize_parallel
from montecarlo import MonteCarlo
from scipy.optimize import differential_evolution
from ECIR_new import ECIRModel

In [17]:
class Calibration:
    def __init__(self, model_class, model_params, optimize_args, n, m, r0, seed=93756826):
        self.n = n
        self.m = m
        self.r0 = r0
        self.seed = seed
        self.model_class = model_class
        self.model_params = model_params
        self.optimize_args = optimize_args

    def _calculate_error(self, optimize_params, Ts, prices):
        model_params = dict(zip(self.optimize_args, optimize_params))
        model = self.model_class(**model_params)
        monte_carlo = MonteCarlo(model, self.r0, max(Ts), self.m, self.n)

        # Compute average prices for each maturity using Monte Carlo simulation
        simulated_prices = []
        for T in Ts:
            monte_carlo.T = T
            avg_price, std_dev, _ = monte_carlo.price_estimates()
            simulated_prices.append(avg_price)

        errors = (simulated_prices - prices) ** 2
        return np.sqrt(np.mean(errors))

    def calibrate(self, bounds, Ts, prices):
        result = differential_evolution(
            lambda params: self._calculate_error(params, Ts, prices),
            bounds=bounds,
            strategy='rand1bin',
            maxiter=100,
            popsize=15,
            tol=0.01,
            mutation=(0.5, 1),
            recombination=0.7,
            disp=True
        )
        return result

In [18]:
if __name__ == "__main__":
    df = pd.read_csv("DGS_30.csv", index_col=0)
    df.index = pd.to_datetime(df.index)
    selected_date = '2024-03-15'
    yields = df.loc[selected_date].astype(float) / 100

    yield_data = pd.DataFrame({
        'Yield': yields,
        'Maturity': np.arange(1, len(yields) + 1)
    })

    prices = np.exp(-yield_data["Yield"] * yield_data["Maturity"])
    Ts = yield_data["Maturity"].values

    initial_model_params = {
        "kappa": 0.5,
        "mu_r": 0.03,
        "sigma": 0.03,
        "mu": 0.05,
        "gamma": 0.01
    }

    bounds = [(0.001, 5), (0.001, 0.1), (0.001, 0.5), (0, 1), (0.001, 0.1)]
    optimize_args = ['kappa', 'mu_r', 'sigma', 'mu', 'gamma']

    calibrator = Calibration(
        ECIRModel, 
        initial_model_params, 
        optimize_args, 
        n=100, 
        m=26, 
        r0=0.001
    )

    optimal_params = calibrator.calibrate(bounds, Ts, prices.values.flatten())
    print(optimal_params)

differential_evolution step 1: f(x)= 0.2204546473003251
differential_evolution step 2: f(x)= 0.2204546473003251
differential_evolution step 3: f(x)= 0.20939557858331442
differential_evolution step 4: f(x)= 0.20939557858331442
differential_evolution step 5: f(x)= 0.20939557858331442
differential_evolution step 6: f(x)= 0.20939557858331442
differential_evolution step 7: f(x)= 0.2079439757389279
differential_evolution step 8: f(x)= 0.2079439757389279
differential_evolution step 9: f(x)= 0.2071401906019873
differential_evolution step 10: f(x)= 0.2071401906019873
differential_evolution step 11: f(x)= 0.2071401906019873
differential_evolution step 12: f(x)= 0.2071401906019873
differential_evolution step 13: f(x)= 0.2069618716250589
differential_evolution step 14: f(x)= 0.2069618716250589
differential_evolution step 15: f(x)= 0.2069618716250589
differential_evolution step 16: f(x)= 0.2069618716250589
differential_evolution step 17: f(x)= 0.2069618716250589
differential_evolution step 18: f(x)

In [ ]:
params = {
    "kappa": 3.071,    
    "mu_r": 0.007668,     
    "sigma": 0.3419,    
    "mu": 0.08226,      
    "gamma": 0.04355            
}